# Apply adjustments - Python

### Overview
Adjusted prices and analytics are commonly used in technical and financial analysis.

This sample demonstrates how to directly request and plot `intraday bars` with a **customizable** time range sampling.<br>

It enables to retrieve **on-demand** data by calling a dedicated bars service on a tick by tick data store. 

### Inputs/outputs
Apply adjustments to tick analytics samples using the request parameter **adjustment** to request:
* Adjusted analytics: set *adjustment* to *True*
* Non-adjusted analytics: set *adjustment* to *False*

To illustrate adjustment application we chose *Tick bars service*. The workflow wi be the same for the other tick analytics services. 

### Services used
This sample uses *gRPC requests* in order to retrieve intraday bars from the dedicated hosted service. The queried endpoint in this script are:
* *TickBarsService*: to directly retrieve tick bars objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *mplfinance* as per display packages
***

# Run intraday bars sample with adjustments

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly mplfinance

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import timedelta
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_bars_pb2 as tick_bars
import systemathics.apis.services.tick_analytics.v1.tick_bars_pb2_grpc as tick_bars_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *tick bars service*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Bars request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AMZN'
exchange = 'XNGS'

#### 3.2 Bars calculation parameters

In [ ]:
# set the bar duration
sampling = 5 * 60

# set the bar calculation field
field = tick_bars.BAR_PRICE_TRADE 

#### 3.3 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
# There is a split (1:20) on 2022-06-06
# So lets choose a date interval to include the split date
today = date.Date(year = 2022, month = 6, day = 7)
start = date.Date(year = 2022, month = 6, day = 3)

date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = start.year, month = start.month, day = start.day), 
    end_date = date.Date(year = today.year, month = today.month, day = today.day)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 15, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 0, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.4 Unadjusted bars request creation
The following code snippet creates *gRPC client*, process request and ensure that the reply is not empty:

In [ ]:
# generate tick bars request
requestUnadjusted = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = my_constraints,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    adjustment = False # do not adjust data (split)
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_bars_service.TickBarsServiceStub(channel)
        
        # process the tick bars request (unadjusted)
        barsUnadjusted = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=requestUnadjusted, metadata=metadata):
                barsUnadjusted.append(bar)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
ticks_count = 0
for b in barsUnadjusted:
    ticks_count += b.count
    
print('{0} tick unadjusted bars retrieved from {1}'.format(len(barsUnadjusted), ticks_count))

#### 3.5 Adjusted bars request creation
The following code snippet creates *gRPC client*, process request and ensure that the reply is not empty:

In [ ]:
# generate tick bars request
requestAdjusted = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = my_constraints,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    adjustment = True # adjust data (split)
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_bars_service.TickBarsServiceStub(channel)
                
        # process the tick bars request (adjusted)
        barsAdjusted = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=requestAdjusted, metadata=metadata):
                barsAdjusted.append(bar)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved bars count:

In [ ]:
ticks_count = 0
for b in barsAdjusted:
    ticks_count += b.count
    
print('{0} tick adjusted bars retrieved from {1}'.format(len(barsUnadjusted), ticks_count))

### Step 4: Retrieve data

#### 4.1 Retrieve intraday unadjusted bars
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content (Unadjusted)
datesUnadjusted=[datetime.fromtimestamp(b.time_stamp.seconds) for b in barsUnadjusted]
opensUnadjusted = [b.open for b in barsUnadjusted]
highsUnadjusted = [b.high for b in barsUnadjusted]
lowsUnadjusted = [b.low for b in barsUnadjusted]
closesUnadjusted = [b.close for b in barsUnadjusted]
volumesUnadjusted = [ts.volume for ts in barsUnadjusted]
countsUnadjusted = [ts.count for ts in barsUnadjusted]

In [ ]:
# create a pandas dataframe with: dates, bars and ticks count used for each bar
d = {'Date': datesUnadjusted, 'Open': opensUnadjusted, 'High': highsUnadjusted, 'Low' : lowsUnadjusted,'Close': closesUnadjusted, 'Volume': volumesUnadjusted, 'Count': countsUnadjusted}
df = pd.DataFrame(data=d)
df = df.set_index('Date')
df

#### 4.2 Retrieve intraday adjusted bars
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
datesAdjusted=[datetime.fromtimestamp(b.time_stamp.seconds) for b in barsAdjusted]
opensAdjusted = [b.open for b in barsAdjusted]
highsAdjusted = [b.high for b in barsAdjusted]
lowsAdjusted = [b.low for b in barsAdjusted]
closesAdjusted = [b.close for b in barsAdjusted]
volumesAdjusted = [ts.volume for ts in barsAdjusted]
countsAdjusted = [ts.count for ts in barsAdjusted]

In [ ]:
da = {'Date': datesAdjusted, 'Open': opensAdjusted, 'High': highsAdjusted, 'Low' : lowsAdjusted,'Close': closesAdjusted, 'Volume': volumesAdjusted, 'Count': countsAdjusted}
dfa = pd.DataFrame(data=da)
dfa = dfa.set_index('Date')
dfa

### Step 5: Visualize data with mplfinance

#### 5.1 Plot unadjusted intraday bars

In [ ]:
import mplfinance as mpf

mpf.plot(df, figratio=(40,15),
         type = 'candle', 
         title=('{0}-min tick bars for {1}-{2} | Before adjustments'.format(int(sampling/60),ticker,exchange)),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

#### 5.2 Plot adjusted intraday bars

In [ ]:
import mplfinance as mpf

mpf.plot(dfa, figratio=(40,15),
         type = 'candle', 
         title=('{0}-min tick bars for {1}-{2} | After adjustments'.format(int(sampling/60),ticker,exchange)),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

### Step 6: Visualize data with plotly

#### 6.1 Plot unadjusted intraday bars

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_layout(title = 'Unadjusted {0}-min bars for {1}-{2}'.format(int(sampling/60), ticker, exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()

#### 6.2 Plot adjusted intraday bars

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Candlestick(x=dfa.index, open=dfa['Open'], high=dfa['High'], low=dfa['Low'], close=dfa['Close'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=dfa.index, y=dfa['Volume'], name="Volume"), secondary_y=False)

fig.update_layout(title = 'Adjusted {0}-min bars for {1}-{2}'.format(int(sampling/60), ticker, exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()

### Step 7: Export data

In [ ]:
dtot = {'Date': datesUnadjusted, 'Open': opensUnadjusted, 'High': highsUnadjusted, 'Low' : lowsUnadjusted,'Close': closesUnadjusted, 'Volume': volumesUnadjusted, 'Count': countsUnadjusted, 'Open_Adj': opensAdjusted, 'High_Adj': highsAdjusted, 'Low_Adj' : lowsAdjusted,'Close_Adj': closesAdjusted, 'Volume_Adj': volumesAdjusted, 'Count_Adj': countsAdjusted}
dftot = pd.DataFrame(data=dtot)
dftot = dftot.set_index('Date')
dftot

In [ ]:
# export csv file
os.makedirs('output', exist_ok=True)
dftot.to_csv('output/{0}-{1}_adjustments.csv'.format(ticker,exchange), index=True)